In [1]:
import selenium

In [8]:
import pickle
import requests
from bs4 import BeautifulSoup as BS

In [3]:
with open("./kbo.pkl", "rb") as f:
    kbo = pickle.load(f)

In [6]:
len(kbo)

829

In [7]:
kbo_player_url = "https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId={}"

In [16]:
cnt = 0
for idx, id_ in enumerate(kbo):
    if idx % 100 == 0:
        print (idx, kbo_player_url.format(id_))

0 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=69158
100 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=50848
200 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=62920
300 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=66160
400 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=51546
500 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=52641
600 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=50040
700 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=77532
800 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=51702


In [17]:
r = requests.get(kbo_player_url.format(id_))

In [21]:
bs = BS(r.text, 'html.parser')

# 선수 각각은 dict 저장, dict -> list 담는다. 

[ {선수1}, {선수2}..... {선수829} ] 

In [32]:
player_dict = dict()
for x in bs.find("div", class_ = "player_basic").findAll("li"):
    a, b = x.text.split(":")
    player_dict[a] = b.strip()

In [33]:
player_dict

{'선수명': '황영국',
 '등번호': 'No.58',
 '생년월일': '1995년 12월 26일',
 '포지션': '투수(좌투좌타)',
 '신장/체중': '185cm/83kg',
 '경력': '청주우암초-청주중-청주고-한화-경찰',
 '입단 계약금': '20000만원',
 '연봉': '3400만원',
 '지명순위': '14 한화 1차',
 '입단년도': '14한화'}

In [37]:
cnt = 0
player_all = list()
for idx, id_ in enumerate(kbo):
    if idx % 30 == 0:
        print(idx)
    r = requests.get(kbo_player_url.format(id_))
    player_dict = dict()
    bs = BS(r.text, 'html.parser')
    for x in bs.find("div", class_ = "player_basic").findAll("li"):
        a, b = x.text.split(":")
        player_dict[a] = b.strip()
    player_all.append(player_dict)

0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810


In [1]:
import pickle
with open("./kbo_detail.pkl", "rb") as f:
    player_all = pickle.load(f)

# KBO 최고 연봉은? (금액)
* 주의사항 : 외국인 선수는 환율 1200원 계산 

In [52]:
total = []
for x in player_all:
#     print(x['연봉'])
    if x['연봉'] == '':
        continue
    연봉 = int(x['연봉'][:-2])
    if x['연봉'][-2:] == "달러":
        연봉 = 연봉 * 1200
    else:
        연봉 = 연봉 * 10000
    total.append(연봉)   

In [54]:
total.sort()

In [56]:
total[-1]

8100000000

In [57]:
for x in player_all:
#     print(x['연봉'])
    if x['연봉'] == '':
        continue
    연봉 = int(x['연봉'][:-2])
    if x['연봉'][-2:] == "달러":
        연봉 = 연봉 * 1200
    else:
        연봉 = 연봉 * 10000
    x['연봉'] = 연봉

In [64]:
maximum = 0
a = dict()
for x in player_all:
    if x['연봉'] != "":
        if x['연봉'] > maximum:
            maximum = x['연봉']
            a['max'] = x

In [4]:
for x in player_all:
    x
    break

In [11]:
x['신장/체중']

'180cm/90kg'

In [12]:
tmp = []
for a in x['신장/체중'].split("/"):
    tmp.append(int(a.replace("cm", "").replace("kg","")))

In [13]:
tmp

[180, 90]

In [15]:
height, weight = [int(a.replace("cm", "").replace("kg","")) for a in x['신장/체중'].split("/")]

In [18]:
bmi = weight / (height / 100) ** 2

In [19]:
bmi

27.777777777777775

In [20]:
for x in player_all:
    height, weight = [int(a.replace("cm", "").replace("kg","")) for a in x['신장/체중'].split("/")]
    x['BMI'] = weight / (height / 100) ** 2

#### BMI 지수가 가장 높은 선수?

In [23]:
maximum = 0
rt = dict()
for x in player_all:
    if maximum < x['BMI']:
        maximum = x['BMI']
        rt['maximum'] = [x['선수명'], x['BMI']]

In [24]:
rt

{'maximum': ['한기원', 39.24646781789639]}

In [27]:
x

{'선수명': '황영국',
 '등번호': 'No.58',
 '생년월일': '1995년 12월 26일',
 '포지션': '투수(좌투좌타)',
 '신장/체중': '185cm/83kg',
 '경력': '청주우암초-청주중-청주고-한화-경찰',
 '입단 계약금': '20000만원',
 '연봉': '3400만원',
 '지명순위': '14 한화 1차',
 '입단년도': '14한화',
 'BMI': 24.251278305332356}

In [28]:
tmp = {}
for x in player_all:
    tmp[x['선수명']] = x['BMI']

In [32]:
sorted(tmp.items(), key=lambda x : x[1], reverse=True)

[('한기원', 39.24646781789639),
 ('이대호', 34.541396535232224),
 ('이성원', 33.30898466033601),
 ('수아레즈', 33.2409972299169),
 ('박상준', 32.82413836636788),
 ('최형우', 32.716049382716044),
 ('한동희', 32.60475787948315),
 ('김승현', 32.407407407407405),
 ('지시완', 32.05030371478282),
 ('김도규', 32.009548611111114),
 ('박진', 32.00096606690013),
 ('진승현', 31.899810964083173),
 ('이태오', 31.855955678670362),
 ('김주현', 31.795583304428252),
 ('나지완', 31.69907016060862),
 ('안승한', 31.63739669421488),
 ('황대인', 31.561671506122963),
 ('나균안', 31.50653254711527),
 ('박병호', 31.26369612856099),
 ('김태연', 31.246054791061734),
 ('조무근', 30.75),
 ('정보근', 30.693877551020407),
 ('안치홍', 30.614821360939274),
 ('푸이그', 30.556813037573566),
 ('서의태', 30.55585078116697),
 ('이승현', 30.457762250291136),
 ('서준원', 30.38714390065741),
 ('폰트', 30.336384869392468),
 ('놀린', 30.336384869392468),
 ('임석진', 30.24691358024691),
 ('주성원', 30.24691358024691),
 ('성동현', 30.234315948601665),
 ('노성호', 30.189590629151066),
 ('김기연', 29.983587930816814),
 ('추신수', 29

* 파생변수 
# 키 -> 나이
# KBO 선수의 평균 나이? 
# 포지션별 등록 선수? 